### Load libraries and files

In [1]:
import pickle
import re
import pandas as pd
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import ftfy
import humanize
import datetime
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
iab_text = pd.read_csv('../Data/IAB_text_v3.csv')

In [3]:
iab_text.head()

,Unique ID,Parent,Name,Position,URL,Title,Snippet,Tier 1,Tier 2,Tier 3,Tier 4,with_bs4,with_justext,with_bs4_clean,with_justext_clean
0,1,-,Automotive,1,https://www.merriam-webster.com/dictionary/aut...,Automotive | Definition of Automotive by …,Automotive definition is - self-propelled. How...,Automotive,-,-,-,Automotive | Defin...,These example sentences are selected automat...,automotive definition automotive merriamwebste...,example sentence select automatically various ...
1,1,-,Automotive,2,https://nl.wikipedia.org/wiki/Automotive,Automotive - Wikipedia,"Dit is een doorverwijspagina, bedoeld om de ve...",Automotive,-,-,-,Automotive - Wikipedia ...,-,NaN,NaN
2,1,-,Automotive,3,https://www.dictionary.com/browse/automotive,Automotive | Definition of Automotive at …,"Automotive definition, pertaining to the desig...",Automotive,-,-,-,Automotive | Definition of Automotive at ...,General Motors warned that a global semicond...,automotive definition automotive dictionarycom...,general motor warn global semiconductor shorta...
3,1,-,Automotive,4,https://en.wikipedia.org/wiki/Automotive_industry,Automotive industry - Wikipedia,The automotive industry comprises a wide range...,Automotive,-,-,-,Automotive industry - Wikipedia ...,The automotive industry began in the 1860s w...,automotive industry wikipedia automotive indus...,automotive industry begin s hundred manufactur...
4,1,-,Automotive,5,https://automotive-online.nl/management/laatst...,Alle nieuwsrubrieken - Automotive Online,Het platform voor het midden en hoger manageme...,Automotive,-,-,-,Alle nieuwsrubrieken - Automotiv...,-,NaN,NaN


In [4]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}

In [5]:
# Cleaning the text sentences so that punctuation marks, stop words &amp; digits are removed
# Words are lemmatized according to their POS tags
def clean(doc):
    doc = ftfy.fix_text(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ""
    for word in punc_free.split():
        tag = nltk.pos_tag([word])[0][1][0].upper()
        normalized = normalized + " " + str(lemma.lemmatize(word, tag_dict.get(tag, wordnet.NOUN)))
    processed = re.sub(r"\d+","", normalized)
    y = processed.split()
    return ' '.join(y)

### With NB

#### On with_bs4 column

In [6]:
df = iab_text.dropna(subset = ['with_bs4_clean'])
df = df[df.with_bs4_clean != 'exceed'].reset_index(drop = True)

tfidf = TfidfVectorizer(max_df = 0.9, min_df = 5)
text_count = tfidf.fit_transform(df['with_bs4_clean'])
y = np.asarray(df[df.columns[7:11]])

[df.columns[7:11]]

[Index(['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4'], dtype='object')]

In [7]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(text_count, y, test_size = 0.25, random_state = 5)

mlb.classes_

array(['-', '3-D Graphics', 'Abkhaz', ..., 'Zulu', 'eSports',
       'the Democratic Republic of the Congo'], dtype=object)

In [8]:
model = MultinomialNB(alpha = 0.1)
model = OneVsRestClassifier(model)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('accuracy_score', metrics.accuracy_score(y_pred, y_test))
print('recall_macro_score', metrics.recall_score(y_pred, y_test, average = 'macro', zero_division = 1))
print('recall_micro_score', metrics.recall_score(y_pred, y_test, average = 'micro', zero_division = 1))
print('recall_weighted_score', metrics.recall_score(y_pred, y_test, average = 'weighted', zero_division = 1))
print('f1_macro_score', metrics.f1_score(y_pred, y_test,average = 'macro', zero_division = 1))
print('f1_micro_score', metrics.f1_score(y_pred, y_test,average = 'micro', zero_division = 1))
print('f1_weighted_score', metrics.f1_score(y_pred, y_test,average = 'weighted', zero_division = 1))
print('precision_macro_score', metrics.precision_score(y_pred, y_test,average = 'macro', zero_division = 1))
print('precision_micro_score', metrics.precision_score(y_pred, y_test,average = 'micro', zero_division = 1))
print('precision_weighted_score', metrics.precision_score(y_pred, y_test,average = 'weighted', zero_division = 1))

accuracy_score 0.007029876977152899
recall_macro_score 0.015688287454711007
recall_micro_score 0.4410202920538593
recall_weighted_score 0.4410202920538593
f1_macro_score 0.013607226524966775
f1_micro_score 0.4568986688933641
f1_weighted_score 0.4331570796908794
precision_macro_score 0.06525714954795744
precision_micro_score 0.4739631101599919
precision_weighted_score 0.45467724039457175


#### On with_justext column

In [9]:
df = iab_text.dropna(subset = ['with_justext_clean'])
df = df[df.with_justext_clean != 'exceed'].reset_index(drop = True)

tfidf = TfidfVectorizer(max_df = 0.9, min_df = 5)
text_count = tfidf.fit_transform(df['with_justext_clean'])
y = np.asarray(df[df.columns[7:11]])

[df.columns[7:11]]

[Index(['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4'], dtype='object')]

In [10]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(text_count, y, test_size = 0.25, random_state = 5)

mlb.classes_

array(['-', '3-D Graphics', 'Abkhaz', ..., 'Zulu', 'eSports',
       'the Democratic Republic of the Congo'], dtype=object)

In [11]:
model = MultinomialNB(alpha = 0.1)
model = OneVsRestClassifier(model)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('accuracy_score', metrics.accuracy_score(y_pred, y_test))
print('recall_macro_score', metrics.recall_score(y_pred, y_test, average = 'macro', zero_division = 1))
print('recall_micro_score', metrics.recall_score(y_pred, y_test, average = 'micro', zero_division = 1))
print('recall_weighted_score', metrics.recall_score(y_pred, y_test, average = 'weighted', zero_division = 1))
print('f1_macro_score', metrics.f1_score(y_pred, y_test,average = 'macro', zero_division = 1))
print('f1_micro_score', metrics.f1_score(y_pred, y_test,average = 'micro', zero_division = 1))
print('f1_weighted_score', metrics.f1_score(y_pred, y_test,average = 'weighted', zero_division = 1))
print('precision_macro_score', metrics.precision_score(y_pred, y_test,average = 'macro', zero_division = 1))
print('precision_micro_score', metrics.precision_score(y_pred, y_test,average = 'micro', zero_division = 1))
print('precision_weighted_score', metrics.precision_score(y_pred, y_test,average = 'weighted', zero_division = 1))

C:\Users\ACER\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\ACER\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 718 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


accuracy_score 0.006261180679785331
recall_macro_score 0.9917894900377412
recall_micro_score 0.8737373737373737
recall_weighted_score 0.8737373737373737
f1_macro_score 0.13705438784219015
f1_micro_score 0.62877911779283
f1_weighted_score 0.8509997694854201
precision_macro_score 0.13424723777543912
precision_micro_score 0.4910967741935484
precision_weighted_score 0.8455128842983122


### Without NB

#### On with_bs4 column

In [12]:
df = iab_text.dropna(subset = ['with_bs4_clean'])
df = df[df.with_bs4_clean != 'exceed'].reset_index(drop = True)

tfidf = TfidfVectorizer()
text_count = tfidf.fit_transform(df['with_bs4_clean'])
y = np.asarray(df[df.columns[7:11]])

In [13]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(text_count, y, test_size = 0.25, random_state = 5)

len(mlb.classes_)

1178

In [14]:
knnClf = KNeighborsClassifier(n_neighbors = 3)
knnClf.fit(x_train, y_train)
knnpred = knnClf.predict(x_test)

print('accuracy_score_KNN', metrics.accuracy_score(knnpred, y_test))
print('recall_macro_score', metrics.recall_score(knnpred, y_test,average = 'macro', zero_division = 1))
print('recall_micro_score', metrics.recall_score(knnpred, y_test,average = 'micro', zero_division = 1))
print('recall_weighted_score', metrics.recall_score(knnpred, y_test,average = 'weighted', zero_division = 1))
print('f1_macro_score', metrics.f1_score(knnpred, y_test,average = 'macro', zero_division = 1))
print('f1_micro_score', metrics.f1_score(knnpred, y_test,average = 'micro', zero_division = 1))
print('f1_weighted_score', metrics.f1_score(knnpred, y_test,average = 'weighted', zero_division = 1))
print('precision_macro_score', metrics.precision_score(knnpred, y_test,average = 'macro', zero_division = 1))
print('precision_micro_score', metrics.precision_score(knnpred, y_test,average = 'micro', zero_division = 1))
print('precision_weighted_score', metrics.precision_score(knnpred, y_test,average = 'weighted', zero_division = 1))

accuracy_score_KNN 0.4344463971880492
recall_macro_score 0.8761127178650618
recall_micro_score 0.862882096069869
recall_weighted_score 0.862882096069869
f1_macro_score 0.5070980486309101
f1_micro_score 0.775858200583352
f1_weighted_score 0.8229451268248577
precision_macro_score 0.48761784522132395
precision_micro_score 0.7047793742993987
precision_weighted_score 0.8069763846158395


#### On with_justext column

In [15]:
df = iab_text.dropna(subset = ['with_justext_clean'])
df = df[df.with_justext_clean != 'exceed'].reset_index(drop = True)

tfidf = TfidfVectorizer()
text_count = tfidf.fit_transform(df['with_justext_clean'])
y = np.asarray(df[df.columns[7:11]])

In [16]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(text_count, y, test_size = 0.25, random_state = 5)

len(mlb.classes_)

1177

In [17]:
knnClf = KNeighborsClassifier(n_neighbors = 3)
knnClf.fit(x_train, y_train)
knnpred = knnClf.predict(x_test)

print('accuracy_score_KNN', metrics.accuracy_score(knnpred, y_test))
print('recall_macro_score', metrics.recall_score(knnpred, y_test,average = 'macro', zero_division = 1))
print('recall_micro_score', metrics.recall_score(knnpred, y_test,average = 'micro', zero_division = 1))
print('recall_weighted_score', metrics.recall_score(knnpred, y_test,average = 'weighted', zero_division = 1))
print('f1_macro_score', metrics.f1_score(knnpred, y_test,average = 'macro', zero_division = 1))
print('f1_micro_score', metrics.f1_score(knnpred, y_test,average = 'micro', zero_division = 1))
print('f1_weighted_score', metrics.f1_score(knnpred, y_test,average = 'weighted', zero_division = 1))
print('precision_macro_score', metrics.precision_score(knnpred, y_test,average = 'macro', zero_division = 1))
print('precision_micro_score', metrics.precision_score(knnpred, y_test,average = 'micro', zero_division = 1))
print('precision_weighted_score', metrics.precision_score(knnpred, y_test,average = 'weighted', zero_division = 1))

accuracy_score_KNN 0.5165474060822898
recall_macro_score 0.8664021912628157
recall_micro_score 0.8561320754716981
recall_weighted_score 0.8561320754716981
f1_macro_score 0.5771287841983385
f1_micro_score 0.7992293931471033
f1_weighted_score 0.8332117125901238
precision_macro_score 0.5947477233491222
precision_micro_score 0.7494193548387097
precision_weighted_score 0.8386795481373311


In [18]:
s = "Jump to navigation NEWSLIVE TV Home APPMAGAZINE HOMEMY FEEDVIDEOSMALAYALAMINDIAGAMINGFACT CHECKQUIZMOVIESHEALTHTECHSPORTSDIU NewsLifestyleYoga can help you keep heart diseases at bay Yoga can help you keep heart diseases at bay Yoga can become a way of life to help you deal with heart diseases. Priyanka Sharma New Delhi June 21, 2018UPDATED: June 21, 2018 18:59 IST Chair posture is proving beneficial for patients who can’t sit on the floor or have disabilities Chair posture is proving beneficial for patients who can’t sit on the floor or have disabilities Even in matters of the heart, literally speaking, yoga has an impact. A study by the All India Institute of Medical Sciences (AIIMS) has established that a person can regulate her/his heartbeats through yoga. AIIMS, in what it claimed to be the first study of its kind to have scientifically established the benefits of yoga, revealed that breathing in a particular rhythm has many positive effects on cardiovascular health. It also helps in overcoming anger, fear, stress and hypertension. Dr KK Deepak, author of the study and head of the Department of Physiology at AIIMS, told Mail Today: A slow yogic breathing can curtail the feeling of stress, fear, anger, tension and also regulate diabetes. The study has found that there is a correlation between heartbeat and blood pressure. Yogic breathing can synchronise the two which is a big deal.  The results of the study, funded by Indian Council for Medical Research (ICMR) and the Ministry of AYUSH, have been published in the latest issue of Indian Journal of Medical Research. When we do deep-breathing, blood pressure comes into the normal range. This indicates that there is a strong relationship between heart rate and blood pressure. If a persons heart rate falls, suddenly the BP will go up and if the BP comes down, the heart rate shoots up. Thus, creating a balance between the two is very important - which can be done by yogic breathing,  Dr Deepak said. In another study conducted by RML Hospital, doctors have found that practising asanas led to significant improvement in patients suffering from mental health illnesses, as compared with those who did other forms of exercise. We made two groups of patients yoga and non-yoga. The yoga group patients cognitive behaviour improved tremendously as compared to the patients doing other exercises, said (Prof) Smita N. Deshpande, Head the Department of Psychiatry & De-Addiction at RML Hospital. Chair Yoga for health Offering relief to patients who cannot sit on floor, Delhi's Sir Ganga Ram Hospital has introduced a new yoga initiative called 'chair yoga' in which the patients can remain seated on a chair and perform health postures. Dr Soina Rawat, director at the department of executive health check up at the hospital, said,A large number of patients were facing problems in doing yoga because of various disabilities. They wanted to practice yoga but couldnt sit on the floor. So we started the concept of chair yoga at our hospital. It is really helpful and their health status has improved. Binda Shukla, 50, is one patient who has been practicing chair yoga for several months. She suffers from upper body stiffness, and has cervical and cholesterol problems. There is definitely relief ever since I started doing meditation and yoga. I feel fresh when I perform deep breathing asanas and am taking less medication now. Even those who have knee joint related complications are comfortable with chair yoga, says Dr Shukla To see whether yoga can improve cognitive functions of patients with severe mental disorder as compared to cardiac controls and to compare improvements in patients with schizophrenia. Time period: Samples of last two years in the study included patients with depression, bipolar disorder, schizophrenia and cardiac controls. Patients were aged between 18-60. Another study was conducted to ascertain randomised control trail on schizophrenia on yoga and non-yoga groups. Exercises: Yoga training included chanting of Om, warm up exercises, breathing (pranayama), various yogic asanas Results: There was an improvement in the speed parameters of most of the cognitive domains among patients with schizophrenia after yoga, compared with the schizophrenia non-yoga group patients. The healthy parameters sustained for six months at least. Tags :Follow YogaFollow International Yoga Day POST A COMMENT READ THIS BJP chief JP Nadda Top BJP leaders take to social media to condemn tweets on farmers' stir by Rihanna, Greta The efficacy of double masking: What health experts have to say Silenced Minority? Airtel, Jio, Vi best prepaid plans with streaming and data benefits under Rs 500 RECOMMENDED WATCH RIGHT NOW 01:00 Watch: Arvind Kejriwal takes first dose of Covid vaccine 02:27 Bengal assembly polls: Mamata Banerjee to file nomination from Nandigram on March 11 00:49 Bengali singer Aditi Munshi joins Trinamool Congress 02:57 Karnataka minister Ramesh Jarkiholi resigns after sex CD scandal 02:01 Good news: Prisoners beautify jail compound in UP's Maharajganj TOP TAKES EC directs petrol pumps to remove hoardings showing PM Modi's photos01:10 EC directs petrol pumps to remove hoardings showing PM Modi's photos BJP calls Rahul Gandhi's push-up challenge violation of code of conduct, writes to EC00:46 BJP calls Rahul Gandhi's push-up challenge violation of code of conduct, writes to EC Delhi Police foils plan to kill 2 Delhi riots accused in Tihar Jail02:30 Delhi Police foils plan to kill 2 Delhi riots accused in Tihar Jail INDIATODAY.IN Download App Andriod AppIOS AppSmartTv App Copyright © 2021 Living Media India Limited. For reprint rights: Syndications Today Covid-19 pandemic: 5 things one should know about heart failure Covid-19 pandemic: 5 things one should know about heart failure KCR's Backward Classes Challenge KCR's Backward Classes Challenge What genes tell us about the risk of developing cancer What genes tell us about the risk of developing cancer Redefining heart health with cutting edge technologies Redefining heart health with cutting edge technologies How yoga can help you to get rid of stress headache How yoga can help you to get rid of stress headache Designer sarees to mark your distinctive impression Designer sarees to mark your distinctive impression Designer sarees to mark your distinctive impression Designer sarees to mark your distinctive impression Do's and dont's to keep in mind while opting for IVF treatment: Here's all you need to know Do's and dont's to keep in mind while opting for IVF treatment: Here's all you need to know Be a style icon with these groovy women handbags Be a style icon with these groovy women handbags India's Avant-Garde India's Avant-Garde"
s = clean(s)
s = tfidf.transform([s])
mlb.inverse_transform(knnClf.predict(s))

[('-',
  'Diseases and Conditions',
  'Heart and Cardiovascular Diseases',
  'Medical Health')]

In [19]:
f = open('../Models/IAB_binarizer.p', 'wb')
pickle.dump(mlb, f)
f.close()

f = open('../Models/IAB_vectorizer.p', 'wb')
pickle.dump(tfidf, f)
f.close()

f = open('../Models/IAB_classifier.p', 'wb')
pickle.dump(knnClf, f)
f.close()

import sklearn
sklearn.__version__

'0.23.2'